In [3]:
# import glassdoor_scraping as gs
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import datetime

In [2]:
# path = "/Users/cristianying/Desktop/ds_salary_proj/scraping-glassdoor-selenium/chromedriver"
# df = gs.get_jobs('data analyst', 15, False, path, 15)

x out worked 1
Scraping terminated before reaching target number of jobs. Needed 15, got 0.


In [4]:
#jobs website
path = "/Users/cristianying/Desktop/ds_salary_proj/scraping-glassdoor-selenium/chromedriver"
driver = webdriver.Chrome(path)
url = 'https://www.glassdoor.com.hk/Job/index.htm'
driver.get(url)
# driver.close() = close the tab .quit close window
#when searching try first: ID, name, class

search_job = driver.find_element_by_id("KeywordSearch")
search_location = driver.find_element_by_id("LocationSearch")
search_job.send_keys("data analyst")
search_location.clear()
search_location.send_keys("Hong Kong")
search_location.send_keys(Keys.RETURN)
jobs = []
now = datetime.datetime.now()
num_jobs = 32
#wait for page to load first before finding the element

while len(jobs) < num_jobs:
    #wait for page to load
    time.sleep(3)
    try:
        driver.find_element_by_xpath(".//li[contains(@class, 'react-job-listing')]").click()
    except ElementClickInterceptedException:
        pass
    #wait to check if sign in pops up
    time.sleep(5) 

    try:
        driver.find_element_by_xpath(".//span[@alt='Close']").click()  #clicking to the X.
    except NoSuchElementException:
        pass
    try:
        job_list = WebDriverWait(driver, 30).until(
            EC.presence_of_all_elements_located((By.XPATH, ".//li[contains(@class, 'react-job-listing')]"))
        )
        for company in job_list:
            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break
            company.click()  #You might 
            collected_successfully = False
            
            #these elements exist for all companies, will wait until page loaded and data is retrieved
            while not collected_successfully:
                try:
                    job_insights = driver.find_element_by_xpath("/html/body/div[3]/div/div/div[1]/div/div[2]/section/div/div/article/div/div[1]/div/div/div[3]/div[2]")
                    
                    company_name = driver.find_element_by_xpath(".//div[@class = 'css-xuk5ye e1tk4kwz5']").text
                    job_type = job_insights.find_element_by_xpath(".//div[1]").text
                    industry = job_insights.find_element_by_xpath(".//div[3]").text
                    company_size = job_insights.find_element_by_xpath(".//div[4]").text
                    collected_successfully = True
                except:
                    #wait until company div shows up
                    time.sleep(1)
            
            #below are separated from the while loop, in some cases they do not exist
            try:
                founded = driver.find_element_by_xpath(".//span[contains(text(), 'Founded')]/following-sibling::span").get_attribute('innerHTML')
            except NoSuchElementException: 
                founded = now.year
            try:
                company_type = driver.find_element_by_xpath(".//span[contains(text(), 'Type') and @class = 'css-1pldt9b e1pvx6aw1']/following-sibling::span").get_attribute('innerHTML')
            except NoSuchElementException: 
                company_type = 'NA'
            try:
                sector = driver.find_element_by_xpath(".//span[contains(text(), 'Sector') and @class = 'css-1pldt9b e1pvx6aw1']/following-sibling::span").get_attribute('innerHTML')
            except NoSuchElementException: 
                sector = 'NA'
            try:
                rating = driver.find_element_by_xpath(".//div[@class = 'mr-sm css-ey2fjr e1pr2f4f3']").text
            except NoSuchElementException: 
                rating = -1
            try:
                base_salary = company.find_element_by_xpath(".//span[@data-test='detailSalary']").text                    
            except NoSuchElementException:
                base_salary = -1 #You need to set a "not found value. It's important."
            
            #add job details to array
            jobs.append(
                {
                    "company_name" : company_name,
                    "job_type" : job_type,
                    "base_salary" : base_salary,
                    "industry" : industry,
                    "company_size" : company_size,
                    "founded" : founded,
                    "rating" : rating,
                    "company_type" : company_type,
                    "sector" : sector
                }
            )
    except BaseException as err:
        print(f"Unexpected {err=}, {type(err)=}")
    try:
        driver.find_element_by_xpath(".//a[@data-test='pagination-next']").click()
    except NoSuchElementException: 
        print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(30, len(jobs)))
        break
        
#     time.sleep(5)
    
#     try:
#         driver.find_element_by_xpath(".//span[@alt='Close']").click()
#     except NoSuchElementException: 
#         print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(30, len(jobs)))
#         break

df_company = pd.DataFrame(jobs)
df_company 



Progress: 0/32
Progress: 1/32
Progress: 2/32
Progress: 3/32
Progress: 4/32
Progress: 5/32
Progress: 6/32
Progress: 7/32
Progress: 8/32
Progress: 9/32
Progress: 10/32
Progress: 11/32
Progress: 12/32
Progress: 13/32
Progress: 14/32
Progress: 15/32
Progress: 16/32
Progress: 17/32
Progress: 18/32
Progress: 19/32
Progress: 20/32
Progress: 21/32
Progress: 22/32
Progress: 23/32
Progress: 24/32
Progress: 25/32
Progress: 26/32
Progress: 27/32
Progress: 28/32
Progress: 29/32
Progress: 30/32
Progress: 31/32
Progress: 32/32


,company_name,job_type,base_salary,industry,company_size,founded,rating,company_type,sector
0,"ExodusPoint Capital Management Hong Kong, Limi...",Job Type: Full-time,-1,Industry: Finance,Size: 501 to 1000 Employees,2017,3.2,Company - Private,Finance
1,Sun Life Hong Kong Ltd\n4.5,Job Type: Full-time,-1,Industry: N/A,Size: Unknown,2021,4.5,Company - Private,NA
2,Achievers Recruitment Limited\n4.7,Job Type: Full-time,-1,Industry: Business Services,Size: 1 to 50 Employees,2016,4.7,Company - Private,Business Services
3,Links International\n4.1,Job Type: Full-time,-1,Industry: Business Services,Size: 51 to 200 Employees,1999,4.1,Company - Private,Business Services
4,lalamove\n3.5,Job Type: Full-time,-1,Industry: Transportation & Logistics,Size: 501 to 1000 Employees,2013,3.5,Company - Private,Transportation &amp; Logistics
5,ION,Job Type: Full-time,-1,Industry: N/A,Size: Unknown,2021,-1,Company - Public,NA
6,The Hong Kong Jockey Club\n3.4,Job Type: Part-time,-1,"Industry: Arts, Entertainment & Recreation",Size: 10000+ Employees,1884,3.4,Non-profit Organisation,"Arts, Entertainment &amp; Recreation"
7,Hong Kong Baptist University\n3.8,Job Type: N/A,-1,Industry: Education,Size: 1001 to 5000 Employees,1956,3.8,College / University,Education
8,The Straits Network\n3.2,Job Type: Full-time,-1,Industry: Information Technology,Size: 201 to 500 Employees,2015,3.2,Company - Private,Information Technology
9,Citi\n3.9,Job Type: N/A,-1,Industry: Finance,Size: 10000+ Employees,1812,3.9,Company - Public,Finance
